In [1]:
import pandas as pd


In [2]:
jp = pd.read_csv('resources/data_scientist_united_states_job_postings_jobspikr.csv')

In [3]:
sl = pd.read_csv('resources/glassdoor.csv')

In [4]:
jp.columns

Index(['crawl_timestamp', 'url', 'job_title', 'category', 'company_name',
       'city', 'state', 'country', 'inferred_city', 'inferred_state',
       'inferred_country', 'post_date', 'job_description', 'job_type',
       'salary_offered', 'job_board', 'geo', 'cursor', 'contact_email',
       'contact_phone_number', 'uniq_id', 'html_job_description'],
      dtype='object')

In [5]:
job_posts = jp[['job_title', 'company_name', 'inferred_city', 'inferred_state',
       'inferred_country', 'post_date', 'salary_offered' ]].copy(deep=True)
job_posts['job_title'].dropna(inplace=True)
job_posts.head()

,job_title,company_name,inferred_city,inferred_state,inferred_country,post_date,salary_offered
0,Enterprise Data Scientist I,Farmers Insurance Group,Woodland hills,California,Usa,2/6/2019,NaN
1,Data Scientist,Luxoft USA Inc,Middletown,New jersey,Usa,2/5/2019,NaN
2,Data Scientist,Cincinnati Bell Technology Solutions,New york,New york,Usa,2/5/2019,NaN
3,"Data Scientist, Aladdin Wealth Tech, Associate...",BlackRock,New york,New york,Usa,2/6/2019,NaN
4,Senior Data Scientist,CyberCoders,Charlotte,North carolina,Usa,2/5/2019,NaN


In [6]:
job_posts['job_title'].str.contains("Data").unique()
job_posts.drop(job_posts[job_posts['job_title'].isna()].index, inplace=True)


job_posts.reset_index(inplace=True, drop=True)
job_posts.head()

,job_title,company_name,inferred_city,inferred_state,inferred_country,post_date,salary_offered
0,Enterprise Data Scientist I,Farmers Insurance Group,Woodland hills,California,Usa,2/6/2019,NaN
1,Data Scientist,Luxoft USA Inc,Middletown,New jersey,Usa,2/5/2019,NaN
2,Data Scientist,Cincinnati Bell Technology Solutions,New york,New york,Usa,2/5/2019,NaN
3,"Data Scientist, Aladdin Wealth Tech, Associate...",BlackRock,New york,New york,Usa,2/6/2019,NaN
4,Senior Data Scientist,CyberCoders,Charlotte,North carolina,Usa,2/5/2019,NaN


In [7]:

job_posts = job_posts[job_posts['job_title'].str.contains("Data Scientist")]

In [8]:
job_posts['job_title'] = "Data Scientist"

In [9]:
job_posts.head()

,job_title,company_name,inferred_city,inferred_state,inferred_country,post_date,salary_offered
0,Data Scientist,Farmers Insurance Group,Woodland hills,California,Usa,2/6/2019,NaN
1,Data Scientist,Luxoft USA Inc,Middletown,New jersey,Usa,2/5/2019,NaN
2,Data Scientist,Cincinnati Bell Technology Solutions,New york,New york,Usa,2/5/2019,NaN
3,Data Scientist,BlackRock,New york,New york,Usa,2/6/2019,NaN
4,Data Scientist,CyberCoders,Charlotte,North carolina,Usa,2/5/2019,NaN


In [10]:
pd.set_option('display.max_rows',None)
# df = pd.DataFrame(job_posts['company_name'].value_counts())
job_posts['company_name'].dropna(inplace=True)

job_posts['company_name'].isna().sum()

1

In [11]:
job_posts.drop(job_posts[job_posts['company_name'].isna()].index, inplace=True)

In [12]:
job_posts['company_name'].isna().sum()

0

In [13]:
To_remove_lst = [' COMPANY INFO FOLLOW GET JOB UPDATES FROM ', 'COMPANY INFO FOLLOW GET JOB UPDATES FROM ', ' .LLC', ' INC.', ' LLC', ' ,INC', ' INC']

job_posts['company_name'] = job_posts['company_name'].str.replace('|'.join(To_remove_lst), '')

job_posts['company_name'] = job_posts['company_name'].str.split(' (\d+)', expand=True)

job_posts['company_name'] = job_posts['company_name'].str.split(',', expand=True)
job_posts['company_name'] = job_posts['company_name'].str.rsplit('REVIEWS', expand=True)

job_posts['company_name'] = job_posts['company_name'].str.upper()




In [14]:
for row in job_posts['company_name']:
    
    l = row.split()
    k = []

    for i in l:

        if (row.count(i)>=1 and (i not in k)):
            k.append(i)
    row = str(' '.join(k))
# job_posts['company_name'] = job_posts['company_name']    
df1 = pd.DataFrame(job_posts['company_name'].value_counts())
df1.reset_index(drop=False, inplace=True)
df1.sort_values(['index']).head()

,index,company_name
1598,DISYS,1
2062,EXPERIS,1
2112,GENESIS10,1
1266,INTELLAPRO,1
639,JEFFERSON FRANK,3


In [15]:
job_posts.to_csv('cleaned_job_posts.csv', index=False)

In [16]:
glassdoor_df = pd.read_csv('cleaned_glassdoor.csv')
glassdoor_df.rename(columns = {'Company Name': 'company_name'}, inplace=True)
glassdoor_df.head()

,company_name,Salary Estimate
0,TECOLOTE RESEARCH,$53K-$91K
1,UNIVERSITY OF MARYLAND MEDICAL SYSTEM,$63K-$112K
2,KNOWBE4,$80K-$90K
3,PNNL,$56K-$97K
4,AFFINITY SOLUTIONS,$86K-$143K


In [17]:
grouped_df = pd.DataFrame(job_posts.groupby(['company_name'])['job_title'].unique())
grouped_df.head()

,job_title
company_name,
DISYS,[Data Scientist]
EXPERIS,[Data Scientist]
GENESIS10,[Data Scientist]
INTELLAPRO,[Data Scientist]
JEFFERSON FRANK,[Data Scientist]


In [18]:
merged_df = pd.merge(grouped_df, glassdoor_df, on='company_name', how='inner')
merged_df

,company_name,job_title,Salary Estimate
0,ADOBE,[Data Scientist],$116K-$208K
1,AUDIBLE,[Data Scientist],$102K-$190K
2,AUTODESK,[Data Scientist],$76K-$140K
3,AVANADE,[Data Scientist],$60K-$114K
4,BRIDG,[Data Scientist],$71K-$135K
5,BWX TECHNOLOGIES,[Data Scientist],$70K-$122K
6,CALIBRE SYSTEMS,[Data Scientist],$50K-$92K
7,CAPGEMINI,[Data Scientist],$110K-$184K
8,CAPTECH,[Data Scientist],$68K-$123K
9,CARILION CLINIC,[Data Scientist],$58K-$104K
